In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
import pickle
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import csv

In [2]:
record_list = [] # Initialize the array that will hold the list of our records

records = os.path.normpath('mit-bih-dataframes/subject_list.csv') # Get our record list like we did in the initial extraction
with open(records) as rfile:# Load our records into the array
    for record in rfile:
        record = record[0:-1] # The -1 removes the newline ("\n") character from the string
        record_list.append(record)

In [5]:
subject_dataframes = {}

for x in tqdm(record_list):
    subject_dataframes[x] = pd.read_csv('mit-bih-subsets/' + x + '_subset_list.csv', index_col=0)

100%|██████████████████████████████████████████| 23/23 [00:00<00:00, 242.18it/s]


In [6]:
print(subject_dataframes['04015'])

            subsetID rhythmLabel
0        04015-0.csv           N
1        04015-1.csv           N
2        04015-2.csv           N
3        04015-3.csv           N
4        04015-4.csv           N
...              ...         ...
1708  04015-1708.csv           N
1709  04015-1709.csv           N
1710  04015-1710.csv           N
1711  04015-1711.csv           N
1712  04015-1712.csv           N

[1713 rows x 2 columns]


In [10]:
rrs = []
for idx, subj in enumerate(record_list):
    rr_ints.append(pd.read_csv(os.path.normpath('mit-bih-extracted/'+x+'_rr_rhythms.csv'), index_col=0))

In [ ]:
#this removes outliers so copy-paste your own code here
#the comment is how I removed outliers
outlier_list=[]
rrs=[]
# for idx, subj in tqdm(enumerate(rr_ints)):
#     outlier = [[],[]] # I want to store the index and the outlier
#     for idx2, rr in enumerate(subj):
#         if rr > 500:
#             outlier[0].append(rr) # add the rr int to our list
#             outlier[1].append(idx2) # Add its index to our list
#     subj = np.delete(subj, outlier[1]) # Remove all found outliers from our subject 
#     rrs.append(subj) # Add it to our new rr interval list
#     outlier_list.append(outlier)

23it [00:00, 37.89it/s]


In [38]:
#creates empty dictionary for subsets
subset_list = {}

for x in tqdm(record_list):
    subset_list[x] = []
    subset_df = pd.read_csv(os.path.normpath('mit-bih-subsets/'+x+'_subset_list.csv'), index_col=0)
    for idx, subsetID in subset_df['subsetID'].items():
        subset = pd.read_csv(os.path.normpath('mit-bih-subsets/'+str(x)+'/'+str(subsetID)), index_col=0)
        subset_list[x].append(subset['rr_int'].tolist())

print(subset_list)

100%|███████████████████████████████████████████| 23/23 [01:15<00:00,  3.28s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




In [40]:
subset_list['04015'][0]

[139,
 158,
 226,
 145,
 144,
 239,
 148,
 149,
 234,
 212,
 217,
 202,
 207,
 206,
 137,
 135,
 131,
 125,
 117,
 220,
 224,
 134,
 146,
 249,
 139]

In [41]:
#Classifying transitions between RR-Ints as short, medium, or long using running mean
int_types = []

for idx, subject in tqdm(enumerate(record_list)):
    #list of types of rr_ints for each subject
    subject_types = []
    for i, subset in enumerate(subset_list[subject]):
        subset_types = []
        for j, rr_ints in enumerate(subset):
            if j==0:
                runningmean = rr_ints
            runningmean = 0.75*runningmean + 0.25*rr_ints
            if rr_ints < 0.85*runningmean: 
                #if rr_ints is less than 85% of runningmean
                #label subject type as short
                subset_types.append('short')
            elif rr_ints > 1.15*runningmean: 
                #if rr_ints is greater than 115% of runningmean
                #label subject type as long
                subset_types.append('long')
            else:
                #label subject type as regular
                subset_types.append('regular')
        subject_types.append(subset_types)
    int_types.append(subject_types)

23it [00:00, 30.39it/s]


In [46]:
int_types[0][0]

['regular',
 'regular',
 'long',
 'regular',
 'regular',
 'long',
 'regular',
 'regular',
 'long',
 'regular',
 'regular',
 'regular',
 'regular',
 'regular',
 'short',
 'short',
 'short',
 'short',
 'short',
 'long',
 'long',
 'short',
 'regular',
 'long',
 'short']

In [43]:
#Classifying transitions between RR-Ints as short, medium, or long using running mean
transitions_list=[]

for idx, subject in enumerate(int_types): 
    subject_transitions = []
    for j, subset in enumerate(subject):
        StoS, StoR, StoL, RtoS, RtoR, RtoL, LtoS, LtoR, LtoL = [0]*9
        count = 0
        for i, rr_int in enumerate(subset):
            if i<len(subset)-1:
                count += 1
                if subset[i]=='short' and subset[i+1]=='short':
                  StoS+=1
                elif subset[i]=='short' and subset[i+1]=='regular':
                  StoR+=1
                elif subset[i]=='short' and subset[i+1]=='long':
                  StoL+=1
                elif subset[i]=='regular' and subset[i+1]=='short':
                  RtoS+=1
                elif subset[i]=='regular' and subset[i+1]=='regular':
                  RtoR+=1
                elif subset[i]=='regular' and subset[i+1]=='long':
                  RtoL+=1
                elif subset[i]=='long' and subset[i+1]=='short':
                  LtoS+=1
                elif subset[i]=='long' and subset[i+1]=='regular':
                  LtoR+=1
                elif subset[i]=='long' and subset[i+1]=='long':
                  LtoL+=1
        subject_transitions.append([StoS/count, StoR/count, StoL/count, RtoS/count, RtoR/count, RtoL/count, LtoS/count, LtoR/count, LtoL/count])
    transitions_list.append(subject_transitions)

In [44]:
print(transitions_list[0][0])

[0.16666666666666666, 0.041666666666666664, 0.041666666666666664, 0.041666666666666664, 0.2916666666666667, 0.16666666666666666, 0.08333333333333333, 0.125, 0.041666666666666664]


In [47]:
#Standard deviation
Std_list = []
for i, subject in enumerate(record_list):
    Standard_devs = []
    for j, subset in enumerate(subset_list[subject]):
        standard_dev = np.std(subset)
        Standard_devs.append(standard_dev)
    Std_list.append(Standard_devs)

In [48]:
#Coefficient of variance(std/mean)
#same formatting as std but divide by mean to obtain CoefOfVar
CoV_list = []
for i, subject in enumerate(record_list):
    Coefficient_vars = []
    for j, subset in enumerate(subset_list[subject]):
        coefficient_var = np.std(subset)/np.mean(subset)
        Coefficient_vars.append(coefficient_var)
    CoV_list.append(Coefficient_vars)

In [49]:
#Range
#same formatting as std but range is maximum of subset minus minimum of subset
Range_list = []
for i, subject in enumerate(record_list):
    Ranges = []
    for j, subset in enumerate(subset_list[subject]):
        range = np.max(subset)-np.min(subset)
        Ranges.append(range)
    Range_list.append(Ranges)

In [50]:
dfdic = {}
for idx, subj in tqdm(enumerate(record_list)):
    dfdic[subj] = pd.DataFrame(
        data = np.transpose(np.array([
                                    [x[0] for x in transitions_list[idx]],
                                    [x[1] for x in transitions_list[idx]],
                                    [x[2] for x in transitions_list[idx]],
                                    [x[3] for x in transitions_list[idx]],
                                    [x[4] for x in transitions_list[idx]],
                                    [x[5] for x in transitions_list[idx]],
                                    [x[6] for x in transitions_list[idx]],
                                    [x[7] for x in transitions_list[idx]],
                                    [x[8] for x in transitions_list[idx]],
                                    Std_list[idx],
                                    CoV_list[idx],
                                    Range_list[idx],
                                    ])
                           ),
        columns = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS', 'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range']
    )


23it [00:00, 146.04it/s]


In [51]:
for idx, x in enumerate(record_list):
    dfdic[x] = pd.concat([subject_dataframes[x], dfdic[x]], axis=1)

In [53]:
dfdic['04015'].head(10)

,subsetID,rhythmLabel,StoS,StoR,StoL,RtoS,RtoR,RtoL,LtoS,LtoR,LtoL,STD,CoefVar,Range
0,04015-0.csv,N,0.166667,0.041667,0.041667,0.041667,0.291667,0.166667,0.083333,0.125000,0.041667,42.632119,0.243167,132.0
1,04015-1.csv,N,0.166667,0.041667,0.083333,0.083333,0.333333,0.083333,0.041667,0.083333,0.083333,52.165797,0.293660,172.0
2,04015-2.csv,N,0.000000,0.041667,0.000000,0.000000,0.666667,0.125000,0.041667,0.083333,0.041667,29.324393,0.149158,102.0
3,04015-3.csv,N,0.250000,0.083333,0.083333,0.041667,0.166667,0.125000,0.166667,0.041667,0.041667,43.730081,0.263117,126.0
4,04015-4.csv,N,0.125000,0.208333,0.000000,0.083333,0.166667,0.208333,0.166667,0.041667,0.000000,44.593704,0.249071,125.0
5,04015-5.csv,N,0.083333,0.083333,0.083333,0.041667,0.125000,0.208333,0.166667,0.125000,0.083333,55.357688,0.316474,181.0
6,04015-6.csv,N,0.083333,0.166667,0.083333,0.083333,0.083333,0.208333,0.208333,0.083333,0.000000,41.390820,0.227923,108.0
7,04015-7.csv,N,0.125000,0.208333,0.041667,0.166667,0.041667,0.166667,0.125000,0.083333,0.041667,51.075849,0.293135,172.0
8,04015-8.csv,N,0.041667,0.250000,0.041667,0.125000,0.041667,0.208333,0.166667,0.083333,0.041667,54.959680,0.320353,175.0
9,04015-9.csv,N,0.166667,0.041667,0.041667,0.083333,0.583333,0.041667,0.000000,0.041667,0.000000,35.555568,0.255722,114.0


In [55]:
reload_flag = False
for idx, x in enumerate(record_list):
    if not os.path.exists('mit-bih-features/'+x+ '.csv') or reload_flag:
        dfdic[x].to_csv('mit-bih-features/'+x+'.csv') 